In [98]:
import pandas as pd
import numpy as np
movies_df=pd.read_csv('movies.csv')
movies_df.drop('title',1,inplace=True)
movies_df['genres']=movies_df.genres.str.split('|')
genres_df=movies_df.copy()
for index,row in movies_df.iterrows():
    for genre in row['genres']:
        genres_df.at[index,genre]=1
genres_df.drop('genres',1,inplace=True)
genres_df.fillna(0,inplace=True)
genres_df.to_csv('genres.csv',index=False)
genres_df=pd.read_csv('genres.csv')


In [99]:
userInput=[
    {'movieId':34,'rating':5.0},
    {'movieId':24,'rating':4.0},
    {'movieId':14,'rating':3.0},
    {'movieId':4,'rating':2.0},
    {'movieId':44,'rating':1.0}
]
userInput=pd.DataFrame(userInput)
userInput

,movieId,rating
0,34,5.0
1,24,4.0
2,14,3.0
3,4,2.0
4,44,1.0


In [100]:
userGenre=genres_df[genres_df['movieId'].isin(userInput['movieId'].tolist())]
userGenre.drop('movieId',1,inplace=True)
userGenre.reset_index(drop=True)
userProfile = userGenre.transpose().dot(userInput.rating.values)
genreTable=genres_df.copy()
genreTable.set_index('movieId',inplace=True)

recommend_df=((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommend_df=((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommend_df.head(5)

movieId
1    0.28125
2    0.12500
3    0.31250
4    0.75000
5    0.15625
dtype: float64

## MongoDB

In [101]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["flick"]
mycol = mydb["reviews"]

### User input

In [102]:
records=[{ "uid": 1, "movieId": 34,"rating":5.0 },
    {"uid": 1,'movieId':24,'rating':4.0},
    {"uid": 1,'movieId':14,'rating':3.0},
    {"uid": 1,'movieId':4,'rating':2.0},
    {"uid": 1,'movieId':44,'rating':1.0}]
x = mycol.insert_many(records)

In [103]:
x.inserted_ids

[ObjectId('5f3302336cf4ae7a2aff50c6'),
 ObjectId('5f3302336cf4ae7a2aff50c7'),
 ObjectId('5f3302336cf4ae7a2aff50c8'),
 ObjectId('5f3302336cf4ae7a2aff50c9'),
 ObjectId('5f3302336cf4ae7a2aff50ca')]

In [104]:
userInput=[]
for x in mycol.find({"uid":1},{ "_id": 0}):
  userInput.append(x)
userInput=pd.DataFrame(userInput)
userInput

,movieId,rating,uid
0,34,5.0,1
1,24,4.0,1
2,14,3.0,1
3,4,2.0,1
4,44,1.0,1


### Genre_data

In [105]:
genre_json=genres_df.to_json(orient='records')
genre_json=eval(genre_json)
genre_data=genre_json
genre_data[0]

{'movieId': 1,
 'Adventure': 1.0,
 'Animation': 1.0,
 'Children': 1.0,
 'Comedy': 1.0,
 'Fantasy': 1.0,
 'Romance': 0.0,
 'Drama': 0.0,
 'Action': 0.0,
 'Crime': 0.0,
 'Thriller': 0.0,
 'Horror': 0.0,
 'Mystery': 0.0,
 'Sci-Fi': 0.0,
 'IMAX': 0.0,
 'Documentary': 0.0,
 'War': 0.0,
 'Musical': 0.0,
 'Western': 0.0,
 'Film-Noir': 0.0,
 '(no genres listed)': 0.0}

In [106]:
genre_json=genres_df.to_json(orient='records')
genre_json=eval(genre_json)
genre_data=genre_json
genre_data[0]

mycol = mydb["genre_data"]
x = mycol.insert_many(genre_data)

In [107]:
mycol = mydb["genre_data"]
data=[]
for x in mycol.find({},{"_id":0}):
    data.append(x)
genres_df=pd.DataFrame(data)
genres_df.head()


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movieId
0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


### User Genre_data

In [108]:
userGenre_json=userGenre.to_json(orient ='records')
userGenre_json=(eval(userGenre_json))
userGenre_data={"uid":1,"genre_data":userGenre_json}

In [109]:
mycol = mydb["user_genre_data"]
x = mycol.insert_one(userGenre_data)

In [110]:
mycol = mydb["user_genre_data"]
data=[]
x=mycol.find_one({"uid":1})
data=x['genre_data']
userGenre=pd.DataFrame(data)
userGenre.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### User Profile data

In [111]:
userProfile_json=eval(userProfile.to_json())
#print(type(userProfile_json))
userProfile_data={"uid":1,"profile_data":userProfile_json}
userProfile_data

{'uid': 1,
 'profile_data': {'Adventure': 1.0,
  'Animation': 0.0,
  'Children': 2.0,
  'Comedy': 5.0,
  'Fantasy': 1.0,
  'Romance': 5.0,
  'Drama': 14.0,
  'Action': 1.0,
  'Crime': 0.0,
  'Thriller': 0.0,
  'Horror': 0.0,
  'Mystery': 0.0,
  'Sci-Fi': 3.0,
  'IMAX': 0.0,
  'Documentary': 0.0,
  'War': 0.0,
  'Musical': 0.0,
  'Western': 0.0,
  'Film-Noir': 0.0,
  '(no genres listed)': 0.0}}

In [112]:
mycol = mydb["user_profile_data"]
x = mycol.insert_one(userProfile_data)

In [113]:
data=[]
x=mycol.find_one({},{"_id":0})
data=x['profile_data']
userProfile=pd.Series(data)
userProfile

Adventure              1.0
Animation              0.0
Children               2.0
Comedy                 5.0
Fantasy                1.0
Romance                5.0
Drama                 14.0
Action                 1.0
Crime                  0.0
Thriller               0.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 3.0
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

### User recommendations

In [114]:
recommendation_json=eval(recommend_df.to_json())
#type(recommendation_json)
recommendation_data={"uid":1,"recommendation_data":recommendation_json}
recommendation_data

{'uid': 1,
 'recommendation_data': {'1': 0.28125,
  '2': 0.125,
  '3': 0.3125,
  '4': 0.75,
  '5': 0.15625,
  '6': 0.03125,
  '7': 0.3125,
  '8': 0.09375,
  '9': 0.03125,
  '10': 0.0625,
  '11': 0.75,
  '12': 0.15625,
  '13': 0.09375,
  '14': 0.4375,
  '15': 0.21875,
  '16': 0.4375,
  '17': 0.59375,
  '18': 0.15625,
  '19': 0.15625,
  '20': 0.625,
  '21': 0.15625,
  '22': 0.4375,
  '23': 0.03125,
  '24': 0.53125,
  '25': 0.59375,
  '26': 0.4375,
  '27': 0.5,
  '28': 0.59375,
  '29': 0.59375,
  '30': 0.4375,
  '31': 0.4375,
  '32': 0.09375,
  '33': 0.1875,
  '34': 0.5,
  '35': 0.59375,
  '36': 0.4375,
  '37': 0.0,
  '38': 0.21875,
  '39': 0.3125,
  '40': 0.4375,
  '41': 0.4375,
  '42': 0.46875,
  '43': 0.4375,
  '44': 0.09375,
  '45': 0.59375,
  '46': 0.59375,
  '47': 0.0,
  '48': 0.65625,
  '49': 0.59375,
  '50': 0.0,
  '51': 0.46875,
  '52': 0.75,
  '53': 0.46875,
  '54': 0.21875,
  '55': 0.4375,
  '56': 0.28125,
  '57': 0.4375,
  '58': 0.75,
  '59': 0.4375,
  '60': 0.125,
  '61': 0.4

In [115]:
mycol = mydb["user_recommendation_data"]
x = mycol.insert_one(recommendation_data)

In [116]:
data=[]
mycol = mydb["user_recommendation_data"]
x=mycol.find_one({},{"_id":0})
data=x['recommendation_data']
recommend_df=pd.Series(data)
recommend_df.head()

1    0.28125
2    0.12500
3    0.31250
4    0.75000
5    0.15625
dtype: float64